In [1]:
#IPython extension to reload modules before executing user code.
#'autoreload' reloads modules automatically before entering the execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['PYTHONHASHSEED'] = '0'
import gpustat

#select the best free GPU on the nvidia card
stats = gpustat.GPUStatCollection.new_query()
ids = map(lambda gpu: int(gpu.entry['index']), stats)
ratios = map(lambda gpu: float(gpu.entry['memory.used'])/float(gpu.entry['memory.total']), stats)
bestGPU = min(zip(ids, ratios), key=lambda x: x[1])[0]
bestGPU = 3

print("setGPU: Setting GPU to: {}".format(bestGPU))
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = str(bestGPU)

'''
#set memory usage to 0.5
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.33
set_session(tf.Session(config=config))'''

setGPU: Setting GPU to: 3


'\n#set memory usage to 0.5\nfrom keras.backend.tensorflow_backend import set_session\nimport tensorflow as tf\nconfig = tf.ConfigProto()\nconfig.gpu_options.per_process_gpu_memory_fraction = 0.33\nset_session(tf.Session(config=config))'

In [3]:
#!pip install -q sklearn
import collections
import numpy as np, numpy
from keract import get_activations, display_activations
import keras
from keras import backend as K
from keras import optimizers, regularizers
from keras.callbacks import LearningRateScheduler
from keras.datasets import cifar10, cifar100 # we can use also cifar100
from keras.layers import Input, BatchNormalization, AveragePooling2D, ZeroPadding2D, LeakyReLU, GlobalAveragePooling2D, Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model, Model
from keras.regularizers import l2
from keras.utils import plot_model, to_categorical
from keras.utils.vis_utils import model_to_dot
import operator 
import matplotlib.pyplot as plt
import scipy
import sklearn
import sklearn.metrics
import sys
#sys.executable
#sys.path
import time
from IPython.display import SVG

Using TensorFlow backend.


In [4]:
def normalize(x, mean, std):
    # This function normalizes inputs for zero mean and unit variance to speed up learning.
    
    # In case std = 0, we add eps = 1e-7
    eps = K.epsilon()
    x = (x-mean)/(std+eps)
    return x
  
def import_cifar(dataset):
    if dataset == 10:
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    elif dataset == 100:
        (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

    # By default, they are uint8 but we need them float to normalize them
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    # Calculating the mean and standard deviation of the training data
    mean = np.mean(x_train, axis=(0, 1, 2, 3))
    std = np.std(x_train, axis=(0, 1, 2, 3))
    
    # Normalizing 
    x_train = normalize(x_train, mean, std)
    x_test = normalize(x_test, mean, std)

    y_train = to_categorical(y_train, num_classes=dataset)
    y_test = to_categorical(y_test,  num_classes=dataset)
    
    return (x_train, y_train), (x_test, y_test)

In [ ]:
# LOAD DATABase
num_classes = 10
(x_train, y_train), (x_test, y_test) = import_cifar(num_classes)

In [ ]:
# Architecture taken from https://github.com/geifmany/cifar-vgg
# Weight decay and Dropout have been removed
# BatchNormalization before activations
def VGG16_Vanilla_beta(input_shape, num_classes):
    model = Sequential([
        #0
        Conv2D(64, (3, 3), padding='same', input_shape=input_shape),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #3
        Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        #7
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #10
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        #14
        Conv2D(256, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #17
        Conv2D(256, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #20
        Conv2D(256, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        #24
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #27
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #30
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        #34
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #37
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #40
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        #45
        Dense(512),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #48
        Dense(num_classes),
        BatchNormalization(scale=False, center=False),
        Activation('softmax')])
    return model

# Architecture taken from https://github.com/geifmany/cifar-vgg
# BatchNormalization before activations
def VGG16_beta(input_shape, num_classes, weight_decay):
    model = Sequential([
        Conv2D(64, (3, 3), padding='same', input_shape=input_shape, kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #Dropout(0.3),
        Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #Dropout(0.4),
        Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #Dropout(0.4),
        Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #Dropout(0.4),
        Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #Dropout(0.4),
        Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #Dropout(0.4),
        Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #Dropout(0.4),
        Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #Dropout(0.4),
        Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        #Dropout(0.5),
        Flatten(),
        Dense(512, kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #Dropout(0.5),
        Dense(num_classes, kernel_regularizer=regularizers.l2(weight_decay)),
        BatchNormalization(scale=False, center=False),
        Activation('softmax')])
    return model

In [ ]:
#model_name = 'modelLAY_65.49%test_94%train_epoch50_0012lr.hdf5'

modelename = 'vgg16_layca_1'
#layca, sgd, wdecay
model_name = '../weights/4/{}/final/weights-final.hdf5'.format(modelename)

In [ ]:
# LOAD MODEL SDG/LAYCA or WD
model = VGG16_Vanilla_beta(input_shape=(32,32,3), num_classes=10)
#model = VGG16_beta(input_shape=(32,32,3), num_classes=10, weight_decay=0.005)

#model.load_weights('weights/{}.hdf5'.format(model_name))
model.load_weights('{}'.format(model_name))

sgd = optimizers.SGD(lr=0.002, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
# EVALUATION
# Final evaluation of the models
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print("%.2f%% : Model Error" % (100-scores[1]*100))

10000/10000 [==============================] - 4s 371us/step
Test loss: 1.0062860960006714
Test accuracy: 0.7412
25.88% : Model Error


# CLUSTERING

In [ ]:
# On regarde l'activation des pairs de neurones pour un echantillon test pris au hasard
Echantillon = []
nombreDImagesDActivation = 1000
choix = np.random.choice(x_test.shape[0], nombreDImagesDActivation)
Echantillon = x_train[choix, :, : , :]

IM = get_activations(model, Echantillon)
# on affiche les noms des differentes couches
print(IM.keys())
model.summary()

dict_keys(['conv2d_4/BiasAdd:0', 'conv2d_11/BiasAdd:0', 'activation_5/Relu:0', 'conv2d_1/BiasAdd:0', 'batch_normalization_15/cond/Merge:0', 'max_pooling2d_3/MaxPool:0', 'activation_10/Relu:0', 'activation_4/Relu:0', 'activation_6/Relu:0', 'batch_normalization_4/cond/Merge:0', 'max_pooling2d_4/MaxPool:0', 'conv2d_2/BiasAdd:0', 'batch_normalization_1/cond/Merge:0', 'max_pooling2d_2/MaxPool:0', 'conv2d_12/BiasAdd:0', 'activation_9/Relu:0', 'activation_12/Relu:0', 'batch_normalization_7/cond/Merge:0', 'batch_normalization_11/cond/Merge:0', 'activation_3/Relu:0', 'conv2d_3/BiasAdd:0', 'activation_8/Relu:0', 'batch_normalization_5/cond/Merge:0', 'batch_normalization_10/cond/Merge:0', 'conv2d_13/BiasAdd:0', 'conv2d_6/BiasAdd:0', 'batch_normalization_12/cond/Merge:0', 'conv2d_5/BiasAdd:0', 'activation_11/Relu:0', 'conv2d_7/BiasAdd:0', 'activation_7/Relu:0', 'flatten_1/Reshape:0', 'activation_14/Relu:0', 'dense_2/BiasAdd:0', 'activation_1/Relu:0', 'conv2d_10/BiasAdd:0', 'conv2d_9/BiasAdd:0', 'b

In [ ]:
# we suppose k = 2 (activé and inactivé)

def AverageDistance(vecteur,vecteurs_without_vecteur):
    nCluster_i = len(vecteurs_without_vecteur)
    distance = 0
    for vect in vecteurs_without_vecteur:
        #distance = distance + scipy.spatial.distance.euclidean(vecteur,vect)
        distance = distance + numpy.linalg.norm(vecteur-vect)
    a = (1/(nCluster_i))*distance
    return a

def AverageDissimilarity(vecteur, vecteurs_from_another_cluster):
    nCluster_j = len(vecteurs_from_another_cluster)
    distance = 0
    for vect in vecteurs_from_another_cluster:
        #distance = distance + scipy.spatial.distance.euclidean(vecteur,vect)
        distance = distance + numpy.linalg.norm(vecteur-vect)
    b = (1/nCluster_j)*distance
    return b

def Silhouette(vecteurs, a, b):
    s = (b-a)/(max(a,b))
    return s

# quid si pas assez de partité ?
def SilhouetteScore(vecteurs_0,vecteurs_1):
    if len(vecteurs_0) == None or len(vecteurs_1) == None or len(vecteurs_0) == 0 or len(vecteurs_1) == 0:
        print('Error: all entries actived or not: Probably a pathologique neuron')
        return -2
    
    SilhouetteScoreSum = 0
    
    if len(vecteurs_0) < 3 :
        SilhouetteScoreSum = SilhouetteScoreSum + 1 #0
    else:
        for i in range(len(vecteurs_0)):
            a = AverageDistance(vecteurs_0[i], vecteurs_0[0:i]+vecteurs_0[(i+1):])
            b = AverageDissimilarity(vecteurs_0[i], vecteurs_1)
            s = Silhouette(vecteurs_0, a, b)
            SilhouetteScoreSum = SilhouetteScoreSum + s
    
    if len(vecteurs_1) < 2 :
        SilhouetteScoreSum = SilhouetteScoreSum + 1 #0
    else:
        for i in range(len(vecteurs_1)):
            a = AverageDistance(vecteurs_1[i], vecteurs_1[0:i]+vecteurs_1[(i+1):])
            b = AverageDissimilarity(vecteurs_1[i], vecteurs_0)
            s = Silhouette(vecteurs_1, a, b)
            SilhouetteScoreSum = SilhouetteScoreSum + s

    SilhouetteScore = SilhouetteScoreSum/(len(vecteurs_0)+len(vecteurs_1))
        
    return SilhouetteScore

In [ ]:
def TakeTheNearest(Vector, Ratio):
    Length = len(Vector)
    DistVector = []
    for i in range(len(Vector)):
        Dist = 0
        for j in range(len(Vector)):
            if i!=j:
                Dist = Dist + scipy.spatial.distance.euclidean(Vector[i],Vector[j])
        #'cosine'
        DistVector.append(Dist)
    #print(len(Vector[0])) # nombre de dimension des activations
    #print(len(DistVector)) # nombre d'activations activé ou non-activé
    
    UltimeVector = zip(DistVector, Vector)
    UltimeVectorList = list(UltimeVector)
    #print(UltimeVectorList)
    #print(UltimeVectorList[0][0]) # 1ere distance totale
    #print(UltimeVectorList[0][1]) # coordonnées de la premiere activation
    #print(UltimeVectorList[:][1]) # toutes les coordonnées
    UltimeVectorList.sort(key=operator.itemgetter(0))
    #print(UltimeVectorList)
    #print(UltimeVectorList[0][0]) # distance 0
    #print(UltimeVectorList[0][1]) # coord 0
    #print(UltimeVectorList[1][0]) # dist 1
    
    Vec = [lis[1] for lis in UltimeVectorList]
    NewVector = Vec[:int(np.floor(Length*Ratio))]
    return NewVector

## Silhouette score mean by layer

In [ ]:
def RUN(Ratio, NeuronsToTest):
    Silhouette_Score_Mean = []
    NbreOfNeurons = []
    for c in range(12):
        print('Couche '+str(c))
        #Layer to test
        couche_name_1 = 'activation_{}/Relu:0'.format(c+1)
        couche_name_2 = 'conv2d_{}/BiasAdd:0'.format(c+2)
        couche_name_3 = 'activation_{}/Relu:0'.format(c+2)

        NeuronsToTestIntoTheLayer = NeuronsToTest

        X_Kernel_size = len(IM[couche_name_2][0,:,:,:])
        Y_Kernel_size = len(IM[couche_name_2][0,0,:,:])
        Z_Kernel_size = len(IM[couche_name_2][0,0,0,:])

        Silhouette_Score_Sum = 0

        for i in range(NeuronsToTestIntoTheLayer):
            print('Neurone '+str(i))

            #Choix du neurone dans la couche
            X = int(numpy.ceil(numpy.random.rand(1)*X_Kernel_size)-1)
            Y = int(numpy.ceil(numpy.random.rand(1)*Y_Kernel_size)-1)
            Z = int(numpy.ceil(numpy.random.rand(1)*Z_Kernel_size)-1)
            #specification de ses bornes
            X_inf = X-1
            Y_inf = Y-1
            X_sup = X+1
            Y_sup = Y+1
            if X == 0: X_inf = 0
            if Y == 0: Y_inf = 0   
            if X == X_Kernel_size-1: X_sup = X_Kernel_size-1
            if Y == Y_Kernel_size-1: Y_sup = Y_Kernel_size-1

            Vecteurs_0 = [] #inactivé
            Vecteurs_1 = [] #activé
            ##Vecteurs_All = []
            ##Labels_All = []

            for j in range(nombreDImagesDActivation):
                if IM[couche_name_3][j,X,Y,Z] == 0.0:
                    Vecteurs_0.append(IM[couche_name_1][j,X_inf:X_sup,Y_inf:Y_sup,0:Z_Kernel_size].flatten())
                    ##Vecteurs_All.append(IM[couche_name_1][j,X_inf:X_sup,Y_inf:Y_sup,0:Z_Kernel_size].flatten())
                    ##Labels_All.append(0)
                elif IM[couche_name_3][j,X,Y,Z] > 0.0:
                    Vecteurs_1.append(IM[couche_name_1][j,X_inf:X_sup,Y_inf:Y_sup,0:Z_Kernel_size].flatten())
                    ##Vecteurs_All.append(IM[couche_name_1][j,X_inf:X_sup,Y_inf:Y_sup,0:Z_Kernel_size].flatten())
                    ##Labels_All.append(1)
                else:
                    print('error !')
                    break

            Vecteurs_0 = TakeTheNearest(Vecteurs_0, Ratio)
            Vecteurs_1 = TakeTheNearest(Vecteurs_1, Ratio)

            S = SilhouetteScore(Vecteurs_0,Vecteurs_1)
            ##S2 = sklearn.metrics.silhouette_score(Vecteurs_All, Labels_All, metric='euclidean') # -> give the same result ! :)

            if S != -2:
                #print('Neuron '+str(i+1)+'/'+str(NeuronsToTestIntoTheLayer)+' has a silhouette score of '+str(S))
                Silhouette_Score_Sum = Silhouette_Score_Sum + S
            else:
                NeuronsToTestIntoTheLayer = NeuronsToTestIntoTheLayer -1

        Silhouette_Score_Mean.append(Silhouette_Score_Sum / NeuronsToTestIntoTheLayer)
        NbreOfNeurons.append(NeuronsToTestIntoTheLayer)
    print(Silhouette_Score_Mean)
    print(np.mean(Silhouette_Score_Mean))
    
    # plot:
    fig = plt.figure(figsize=(10, 5))
    plt.ylim(top=1)
    nombreCouches = len(Silhouette_Score_Mean)
    x = range(nombreCouches)

    markerline, stemlines, baseline = plt.stem(x, Silhouette_Score_Mean, markerfmt='o', label='model ')
    plt.setp(stemlines, 'color', plt.getp(markerline,'color'))
    plt.setp(stemlines, 'linestyle', 'dotted')
    plt.plot(np.mean(Silhouette_Score_Mean)*np.ones((nombreCouches,1)))
    for c in range(12):
        plt.text(-0.27+c, 0.02, NbreOfNeurons[c], rotation= 90)

    plt.legend()
    plt.xlabel('Couches')
    plt.xticks(np.arange(nombreCouches), ('Conv 1','Conv 2','Conv 3','Conv 4','Conv 5','Conv 6','Conv 7','Conv 8','Conv 9','Conv10','Conv11','Conv12','Conv13','Dense1','Dense2'), rotation=60)
    plt.ylabel('Silhouette score mean')
    plt.title('Silhouette score mean for the {} by layer for the model {}\n for {} activation images and {} neurons'.format(Ratio,modelename,nombreDImagesDActivation,NeuronsToTest))
    plt.show()
    fig.savefig('figures/clustering/SilhouetteScoreMean_{}_Ratio{}.png'.format(modelename, Ratio), dpi=100)

In [ ]:
for j in range(5):
    ratio = round(0.05+0.2*np.round(j), 2)
    print(ratio)
    RUN(Ratio=ratio, NeuronsToTest=10)
    

0.05
Couche 0
Neurone 0
Neurone 1
Neurone 2
Neurone 3
Neurone 4
Neurone 5
Neurone 6
Neurone 7
Neurone 8
Neurone 9
Couche 1
Neurone 0
Neurone 1
Neurone 2
Neurone 3
Neurone 4
Neurone 5
Neurone 6
Neurone 7
Neurone 8
Neurone 9
Couche 2
Neurone 0
Neurone 1
Neurone 2
Neurone 3
Neurone 4
Neurone 5
Neurone 6
Neurone 7
Neurone 8
Neurone 9
Couche 3
Neurone 0
Neurone 1
Neurone 2
Neurone 3
Neurone 4
Neurone 5
Neurone 6
Neurone 7
Neurone 8
Neurone 9
Couche 4
Neurone 0
Neurone 1
Neurone 2
Neurone 3
Neurone 4
Neurone 5
Neurone 6
Neurone 7
Neurone 8
Neurone 9
Couche 5
Neurone 0
Neurone 1
Neurone 2
Neurone 3
Neurone 4
Neurone 5
Neurone 6
Neurone 7
Neurone 8
Neurone 9
Couche 6
Neurone 0
Neurone 1
Neurone 2
Neurone 3
Neurone 4
Neurone 5
Neurone 6
Neurone 7
Neurone 8
Neurone 9
Couche 7
Neurone 0
Neurone 1
Neurone 2
Neurone 3
Neurone 4
Neurone 5
Neurone 6
Neurone 7
Neurone 8
Neurone 9
Couche 8
Neurone 0
Neurone 1
Neurone 2
Neurone 3
Neurone 4
Neurone 5
Neurone 6
Neurone 7
Neurone 8
Neurone 9
Couche 9
Neuro